# Open-AI based chat serivce

In [1]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content


In [2]:
print(get_completion("How are you today? answer in a few words."))

I'm an AI, so I don't have feelings, but I'm here to help you!


In [3]:
from openai import OpenAI
client = OpenAI()

def get_transcript(audio_file_path):
    audio_file = open(audio_file_path, "rb")
    transcript = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file, 
    response_format="text"
    )
    return transcript

def text_to_speech(text, output_file_path, voice="alloy"):
    response = client.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=text
    )
    response.stream_to_file(output_file_path)
    return output_file_path

In [ ]:
test_sentence = "Hello, how are you?"
text_to_speech(test_sentence, "recordings/assistant/audio.mp3")

get_transcript("recordings/assistant/audio.mp3")

In [5]:
import pyaudio
import wave

def record_audio(file_name, duration=5, rate=44100, chunk=1024, format=pyaudio.paInt16, channels=1):
    audio = pyaudio.PyAudio()

    stream = audio.open(format=format,
                        channels=channels,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

    print("Recording...")

    frames = []

    for i in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wave_file = wave.open(file_name, 'wb')
    wave_file.setnchannels(channels)
    wave_file.setsampwidth(audio.get_sample_size(format))
    wave_file.setframerate(rate)
    wave_file.writeframes(b''.join(frames))
    wave_file.close()

In [ ]:
record_audio("audio.wav", duration=10)

In [6]:
# Let's put it all together
# First, we'll record audio to a file
# Then, we'll transcribe the audio
# Finally, we'll send the transcript to the LLM

import datetime
import os
from playsound import playsound


# Get the current time
current_time = datetime.datetime.now()

# Format the current time as a string
filename = current_time.strftime("%Y%m%d-%H%M%S") + ".wav"
dir = "recordings/user"
rec_full_path = os.path.join(dir, filename)

if not os.path.exists(dir):
    os.makedirs(dir)

dir_assistant = "recordings/assistant"
if not os.path.exists(dir_assistant):
    os.makedirs(dir_assistant)

# Record audio to the file
record_audio(rec_full_path, duration=5)

# Transcribe the audio
user_query_text = get_transcript(rec_full_path)
print("user:", user_query_text)

# Send it to the LLM see what it says
response_assistant_text = get_completion(user_query_text)
print("assistant:", response_assistant_text)

# Convert the response to speech
response_assitant_filename = current_time.strftime("%Y%m%d-%H%M%S") + ".mp3"
response_assistant_full_path = os.path.join(dir_assistant, response_assitant_filename)

audio_response_assistant = text_to_speech(response_assistant_text, response_assistant_full_path)

# Play the response
playsound(audio_response_assistant)


playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2666:(snd_pcm_open_nou

Recording...
Finished recording.
user: This is the last experiment I'm doing today.

assistant: I have completed all the experiments scheduled for today, and this is the final one.
